In [ ]:
import pandas as pd

# File template
#file_template = "separate result/results_P_sup_fold{}_desc.xlsx"

file_template = "separate result/results_P_fold{}_reas.xlsx"

#file_template = "separate result/results_P_fold{}_reas_ref.xlsx"



folds = range(5)

# Dictionary to collect accuracy per P across folds
accuracy_by_P = {}

# Load accuracy values from each fold
for fold in folds:
    df = pd.read_excel(file_template.format(fold))

    for _, row in df.iterrows():
        P = row['P']  # assuming 'P' column exists
        acc = row['Overall Accuracy']  # assuming 'overall_accuracy' column exists

        if P not in accuracy_by_P:
            accuracy_by_P[P] = []
        accuracy_by_P[P].append(acc)

# Compute average accuracy per P
avg_accuracy = {
    'P': [],
    'average_overall_accuracy': []
}

for P, acc_list in accuracy_by_P.items():
    avg_accuracy['P'].append(P)
    avg_accuracy['average_overall_accuracy'].append(sum(acc_list) / len(acc_list))

# Convert to DataFrame and save
avg_df = pd.DataFrame(avg_accuracy)
#avg_df.to_excel("separate result/average_overall_accuracy_by_P_desc.xlsx", index=False)

avg_df.to_excel("separate result/average_overall_accuracy_by_P_reas.xlsx", index=False)

#avg_df.to_excel("separate result/average_overall_accuracy_by_P_reas_ref.xlsx", index=False)



In [ ]:
import pandas as pd

# File template
file_template = "separate result/results_P_sup_fold{}_desc.xlsx"


#file_template = "separate result/results_P_fold{}_reas_ref.xlsx"



folds = range(5)



# Dictionary to collect accuracy per P across folds
accuracy_by_P = {}

# Load accuracy values from each fold
for fold in folds:
    df = pd.read_excel(file_template.format(fold))

    for _, row in df.iterrows():
        P = row['P']  # assuming 'P' column exists
        acc = row['Overall Accuracy']  # assuming 'overall_accuracy' column exists

        if P not in accuracy_by_P:
            accuracy_by_P[P] = []
        accuracy_by_P[P].append(acc)

# Compute average accuracy per P
avg_accuracy = {
    'P': [],
    'average_overall_accuracy': []
}

for P, acc_list in accuracy_by_P.items():
    avg_accuracy['P'].append(P)
    avg_accuracy['average_overall_accuracy'].append(sum(acc_list) / len(acc_list))

# Convert to DataFrame and save
avg_df = pd.DataFrame(avg_accuracy)
avg_df.to_excel("separate result/average_overall_accuracy_by_P_desc.xlsx", index=False)

# avg_df.to_excel("separate result/average_overall_accuracy_by_P_reas.xlsx", index=False)

#avg_df.to_excel("separate result/average_overall_accuracy_by_P_reas_ref.xlsx", index=False)



In [ ]:
import pandas as pd

# File template
# file_template = "separate result/results_P_sup_fold{}_desc.xlsx"


file_template = "separate result/results_P_fold{}_reas_ref.xlsx"



folds = range(5)


# Dictionary to collect accuracy per P across folds
accuracy_by_P = {}

# Load accuracy values from each fold
for fold in folds:
    df = pd.read_excel(file_template.format(fold))

    for _, row in df.iterrows():
        P = row['P']  # assuming 'P' column exists
        acc = row['Overall Accuracy']  # assuming 'overall_accuracy' column exists

        if P not in accuracy_by_P:
            accuracy_by_P[P] = []
        accuracy_by_P[P].append(acc)

# Compute average accuracy per P
avg_accuracy = {
    'P': [],
    'average_overall_accuracy': []
}

for P, acc_list in accuracy_by_P.items():
    avg_accuracy['P'].append(P)
    avg_accuracy['average_overall_accuracy'].append(sum(acc_list) / len(acc_list))

# Convert to DataFrame and save
avg_df = pd.DataFrame(avg_accuracy)

avg_df.to_excel("separate result/average_overall_accuracy_by_P_reas_ref.xlsx", index=False)



In [ ]:
import pandas as pd
import os


for fold in range(5):
   
    suffix = f"fold{fold}"


    
    # ---------------- 1. Load Training Sets ----------------
    prob_train_df = pd.read_excel(f"../ref/model_prediction_probabilities_{suffix}_reas_train.xlsx")
    weighted_mse_train_df = pd.read_excel(f"../reas/mse_difference_reasoningUS_{suffix}_reas_train.xlsx")
    mse_train_df = pd.read_excel(f"../desc/optimal_c_and_mse_lstsq_with_name_{suffix}_desc_train.xlsx")
    

    
    # ---------------- 2. Clean video_name columns ----------------
    for df in [prob_train_df, weighted_mse_train_df, mse_train_df]:
        df["video_name"] = df["video_name"].str.replace(".jpg", "", regex=False)
    
    # ---------------- 3. Define (X-mean)/sigma Normalization Function ----------------
    def mean_std_norm(series):
        mean = series.mean()
        std = series.std()
        if std == 0:
            return pd.Series(0.0, index=series.index)
        return (series - mean) / std
    
    # ---------------- 4. Normalize Columns ----------------
    prob_train_df["S3"] = mean_std_norm(prob_train_df["average_prob"])
    weighted_mse_train_df["S2"] = mean_std_norm(weighted_mse_train_df["MSE_difference"])
    mse_train_df["S1"] = mean_std_norm(mse_train_df["MSE"])
    
    # ---------------- 5. Merge into One DataFrame ----------------
    merged_df = mse_train_df[["video_name", "S1"]].merge(
        weighted_mse_train_df[["video_name", "S2"]], on="video_name"
    ).merge(
        prob_train_df[["video_name", "S3"]], on="video_name"
    )
    
    # ---------------- 6. Save Combined File ----------------
    out_dir = "low list"
    # os.makedirs(out_dir, exist_ok=True)
    
    merged_df.to_excel(os.path.join(out_dir, f"combined_normalized_training_sets_meansigma_{suffix}.xlsx"), index=False)
    

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

# Paths and parameters




for fold in range(5):

    suffix = f"fold{fold}"
    
    
    uncertainty_file = f'low list/combined_normalized_training_sets_meansigma_{suffix}.xlsx'
    
    
    model_names = [
    "claude3.5sonnet",
    "claude3.5",
    "claude3.7",
    "gpt4o-mini",
    "gpt4o"
]
    P_SET = [5, 10, 15, 20, 25, 30, 35, 40]
    
    # Load uncertainty scores
    uncertainty_df = pd.read_excel(uncertainty_file)
    uncertainty_df = uncertainty_df.rename(columns={'S1': 'S_data', 'S2': 'S_task', 'S3': 'S_ref'})
    uncertainty_df['video_name'] = uncertainty_df['video_name'].str.replace('.jpg', '', regex=False)
    uncertainty_df = uncertainty_df.set_index('video_name')
    
    # Load all models' predictions and ground truth
    preds = []
    for model in model_names:
        df = pd.read_csv(f'../model_predictions/vad_results_{model}.csv')
        df['Video Name'] = df['Video Name'].str.replace('.jpg', '', regex=False)
        df = df.set_index('Video Name')
        preds.append(df[['Predicted Label']].rename(columns={'Predicted Label': model}))
    merged_preds_df = pd.concat(preds, axis=1)
    merged_preds_df['majority_vote'] = merged_preds_df.mode(axis=1)[0]
    ground_truth = pd.read_csv('../model_predictions/vad_results_claude3.5sonnet.csv')
    ground_truth['Video Name'] = ground_truth['Video Name'].str.replace('.jpg', '', regex=False)
    merged_preds_df['True_Label'] = ground_truth.set_index('Video Name').loc[merged_preds_df.index]['True Label']
    
    # Merge uncertainty with predictions and ground truth
    combined_df = uncertainty_df.join(merged_preds_df.dropna(subset=['True_Label', 'majority_vote']), how='inner')
    
    # Grid search over alpha combinations and P values
    alpha_grid = np.linspace(0, 1, 11)
    results = []
    
    for p_val in P_SET:
        for alpha1 in alpha_grid:
            for alpha2 in alpha_grid:
                if alpha1 + alpha2 > 1.0:
                    continue
                alpha3 = 1.0 - alpha1 - alpha2
                s_total = (alpha1 * combined_df['S_data'] +
                           alpha2 * combined_df['S_task'] +
                           alpha3 * combined_df['S_ref'])
                tau = np.percentile(s_total, 100 - p_val)
                low_uncertainty_df = combined_df[s_total <= tau]
    
                if not low_uncertainty_df.empty:
                    acc = accuracy_score(low_uncertainty_df['True_Label'], low_uncertainty_df['majority_vote'])
                    results.append({
                        'P': p_val,
                        'alpha1': round(alpha1, 2),
                        'alpha2': round(alpha2, 2),
                        'alpha3': round(alpha3, 2),
                        'accuracy': round(acc, 4),
                        'num_videos_kept': len(low_uncertainty_df)
                    })
    
    results_df = pd.DataFrame(results)
    results_df.to_excel(f's_total_accuracy_over_P_and_x_real_labels_meansigma_train_{suffix}.xlsx', index=False)
    
 


In [ ]:
import pandas as pd



for fold in range(5):
    suffix = f"fold{fold}"


    
    # --- File paths ---
    ref_file  = f"../ref/low list/VAD_Majority_Voting_Summary_{suffix}_reas_ref_train.xlsx"
    reas_file = f"../reas/low list/VAD_Majority_Voting_Summary_{suffix}_reas_train.xlsx"
    desc_file = f"../desc/low list/VAD_Majority_Voting_Summary_sup_{suffix}_desc_train.xlsx"
    

    
    # --- Read P and Accuracy, rename Accuracy columns ---
    ref_df  = pd.read_excel(ref_file,  usecols=["P", "Accuracy"]).rename(columns={"Accuracy": "Accuracy_s3"})
    reas_df = pd.read_excel(reas_file, usecols=["P", "Accuracy"]).rename(columns={"Accuracy": "Accuracy_s2"})
    desc_df = pd.read_excel(desc_file, usecols=["P", "Accuracy"]).rename(columns={"Accuracy": "Accuracy_s1"})
    
    # --- Merge on P ---
    merged_df = ref_df.merge(reas_df, on="P").merge(desc_df, on="P")
    
    # --- Save to a new Excel file ---
    merged_df.to_excel(f"low list/VAD_Majority_Voting_P_Accuracy_Combined_{suffix}_train.xlsx", index=False)
    



In [ ]:
import pandas as pd
import numpy as np
import os

# Define folds

folds = [f'fold{i}' for i in range(5)]

# Store all results
all_results = {}

# Process each fold
for fold in folds:


    
    # Construct file paths
    acc_file = f's_total_accuracy_over_P_and_x_real_labels_meansigma_train_{fold}.xlsx'
    base_file = f'low list/VAD_Majority_Voting_P_Accuracy_Combined_{fold}_train.xlsx'

    # Load data
    accuracy_df = pd.read_excel(acc_file)
    baseline_df = pd.read_excel(base_file)

    # Merge by P
    merged_df = pd.merge(accuracy_df, baseline_df[['P', 'Accuracy_s1', 'Accuracy_s2', 'Accuracy_s3']], on='P', how='left')

    # Compute min difference
    merged_df['min_diff'] = merged_df.apply(
        lambda row: min(row['accuracy'] - row['Accuracy_s1'],
                        row['accuracy'] - row['Accuracy_s2'],
                        row['accuracy'] - row['Accuracy_s3']), axis=1)

    # Group and average
    grouped = merged_df.groupby(['P', 'alpha1', 'alpha2', 'alpha3'])['min_diff'].mean().reset_index()

    # Store per (P, alpha1, alpha2, alpha3)
    for _, row in grouped.iterrows():
        key = (row['P'], row['alpha1'], row['alpha2'], row['alpha3'])
        if key not in all_results:
            all_results[key] = []
        all_results[key].append(row['min_diff'])

# Aggregate across folds
aggregated_results = []
for (P, a1, a2, a3), values in all_results.items():
    aggregated_results.append({
        'P': P,
        'alpha1': a1,
        'alpha2': a2,
        'alpha3': a3,
        'mean_min_diff': np.mean(values)
    })

aggregated_df = pd.DataFrame(aggregated_results)

# Get best alpha per P
best_alpha_per_P = aggregated_df.loc[aggregated_df.groupby('P')['mean_min_diff'].idxmax()].reset_index(drop=True)

# Save result
best_alpha_per_P.to_excel('best_alpha_combination_for_each_P_meansigma_train_all_folds.xlsx', index=False)
print("✅ Saved to best_alpha_combination_for_each_P_meansigma_train_all_folds.xlsx")


In [ ]:

import pandas as pd
import os


for fold in range(5):

    suffix = f"fold{fold}"


    
    # Load source files again
    prob_df = pd.read_excel(f"../ref/model_prediction_probabilities_{suffix}_reas.xlsx")
    weighted_mse_df = pd.read_excel(f"../reas/mse_difference_reasoningUS_{suffix}_reas.xlsx")
    mse_df = pd.read_excel(f"../desc/optimal_c_and_mse_lstsq_with_name_{suffix}_desc.xlsx")


    prob_train_df = pd.read_excel(f"../ref/model_prediction_probabilities_{suffix}_reas.xlsx")
    weighted_mse_train_df = pd.read_excel(f"../reas/mse_difference_reasoningUS_{suffix}_reas.xlsx")
    mse_train_df = pd.read_excel(f"../desc/optimal_c_and_mse_lstsq_with_name_{suffix}_desc.xlsx")



    # Clean video_name columns
    for df in [prob_df, weighted_mse_df, mse_df, prob_train_df, weighted_mse_train_df, mse_train_df]:
        df['video_name'] = df['video_name'].str.replace('.jpg', '', regex=False)
    
    # Compute mean and sigma from training sets
    prob_mean = prob_train_df['average_prob'].mean()
    prob_sigma = prob_train_df['average_prob'].std()
    mse_diff_mean = weighted_mse_train_df['MSE_difference'].mean()
    mse_diff_sigma = weighted_mse_train_df['MSE_difference'].std()
    mse_mean = mse_train_df['MSE'].mean()
    mse_sigma = mse_train_df['MSE'].std()
    
    # Merge files
    merged_df = prob_df.merge(weighted_mse_df[['video_name', 'MSE_difference']], on='video_name', how='left')
    merged_df = merged_df.merge(mse_df[['video_name', 'MSE']], on='video_name', how='left')
    
    # Normalize using (X-mean) / sigma
    merged_df['normalized_prob'] = (merged_df['average_prob'] - prob_mean) / prob_sigma
    merged_df['normalized_MSE_difference'] = (merged_df['MSE_difference'] - mse_diff_mean) / mse_diff_sigma
    merged_df['normalized_MSE'] = (merged_df['MSE'] - mse_mean) / mse_sigma
    
    # Load best alpha combinations
    # best_alpha_df = pd.read_excel("best_alpha_combination_for_each_P_meansigma_train.xlsx")
    
    
    best_alpha_df = pd.read_excel("best_alpha_combination_for_each_P_meansigma_train_all_folds.xlsx")
    
    
    # Output with different sheets for each P
    output_file = f"combined_sum_output_with_best_alpha_per_P_{suffix}.xlsx"
    
    #output_file = "combined_sum_output_with_best_alpha_per_P_val.xlsx"
    
    #os.makedirs("adaptive weight", exist_ok=True)
    
    with pd.ExcelWriter(output_file) as writer:
        for _, row in best_alpha_df.iterrows():
            P = row["P"]
            alpha1 = row["alpha1"]
            alpha2 = row["alpha2"]
            alpha3 = row["alpha3"]
    
            merged_df[f'sum_value_P{P}'] = (
                alpha1 * merged_df['normalized_MSE'] +
                alpha2 * merged_df['normalized_MSE_difference'] +
                alpha3 * merged_df['normalized_prob']
            )
    
            merged_df[['video_name', 'normalized_MSE', 'normalized_MSE_difference',
                       'normalized_prob', f'sum_value_P{P}']].to_excel(writer, sheet_name=f"P_{P}", index=False)
    
    print(f"✅ Output saved to {output_file}")



In [ ]:
import pandas as pd
import re

# Define file template and folds
file_template = "combined_sum_output_with_best_alpha_per_P_fold{}.xlsx"


folds = list(range(5))  # You can change to range(5) if needed



# Load sheet names from the first file
sheet_names = pd.ExcelFile(file_template.format(0)).sheet_names

# Value column suffixes to process
value_suffixes = [
    "sum_value_P{}", 
    "normalized_MSE", 
    "normalized_MSE_difference", 
    "normalized_prob"
]

# Dictionary to store results by sheet
all_avg_dfs = {}

# Process each sheet (P value) independently
for sheet in sheet_names:
    # Extract P value for constructing value columns
    match = re.search(r"P_(\d+\.?\d*)", sheet)
    if not match:
        continue
    p_value = match.group(1)
    value_cols = [suffix.format(p_value) for suffix in value_suffixes]

    # Initialize dictionary to collect values for each metric
    video_values = {col: {} for col in value_cols}

    for fold in folds:
        file_path = file_template.format(fold)
        df = pd.read_excel(file_path, sheet_name=sheet)

        for value_col in value_cols:
            if value_col not in df.columns:
                continue

            for _, row in df.iterrows():
                video = row['video_name']
                value = row[value_col]
                if video not in video_values[value_col]:
                    video_values[value_col][video] = []
                video_values[value_col][video].append(value)

    # Create DataFrame with average values
    avg_data = {'video_name': list(set(v for d in video_values.values() for v in d))}
    for value_col in value_cols:
        averages = []
        for video in avg_data['video_name']:
            values = video_values[value_col].get(video, [])
            clean_values = [v for v in values if pd.notnull(v)]
            avg = sum(clean_values) / len(clean_values) if clean_values else None
            averages.append(avg)
        avg_data[value_col] = averages

    avg_df = pd.DataFrame(avg_data)
    all_avg_dfs[sheet] = avg_df

# Save all results to one Excel file
with pd.ExcelWriter("average_video_values_by_P_all.xlsx") as writer:
    for sheet_name, df in all_avg_dfs.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)


In [ ]:
import pandas as pd
import numpy as np
import os

# File path to the average results
file_path = "average_video_values_by_P_all.xlsx"


# Define the P values you want to process
P_set = [5, 10, 15, 20, 25, 30, 35, 40]

# Make output directory
os.makedirs("adaptive weight", exist_ok=True)

# Columns to process per P
column_suffixes = [
    "sum_value_P{}.0",
    "normalized_MSE",
    "normalized_MSE_difference",
    "normalized_prob"
]

for P in P_set:
    sheet_name = f"P_{P}.0"

    try:
        # Read the sheet
        sheet_df = pd.read_excel(file_path, sheet_name=sheet_name)
    except ValueError:
        print(f"Sheet {sheet_name} not found in {file_path}. Skipping.")
        continue

    video_names = sheet_df['video_name'].astype(str).values

    for suffix in column_suffixes:
        col_name = suffix.format(P)
        if col_name not in sheet_df.columns:
            print(f"Column {col_name} not found in sheet {sheet_name}. Skipping.")
            continue

        uncertainty_scores = sheet_df[col_name].values

        # Compute threshold
        tau = np.percentile(uncertainty_scores, 100 - P)
        mask_low = uncertainty_scores <= tau
        S_low_videos = video_names[mask_low]

        out_df = pd.DataFrame({
            "video_name": S_low_videos,
            "uncertainty": uncertainty_scores[mask_low]
        })

        # Save result
        out_filename = f"adaptive weight/S_low_videos_{P}_{col_name}_average.xlsx"
        out_df.to_excel(out_filename, index=False)

        print(f"[P = {P}, Column = {col_name}] Threshold τ = {tau:.4f}, Kept {len(S_low_videos)}/{len(video_names)} videos.")


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Model names
model_names = [
    "claude3.5sonnet",
    "claude3.5",
    "claude3.7",
    "gpt4o-mini",
    "gpt4o"
]
# P values representing percent of videos excluded as high-uncertainty
P_set = [5, 10, 15, 20, 25, 30, 35, 40]




# Store results for each P
all_results = []

for P in P_set:
    
    try:
        low = pd.read_excel(f'low list/S_low_videos_{P}_normalized_meansigma_bestalpha_average_5.xlsx')

    except Exception as e:
        print(f"[Error] Failed to load S_low_videos_{P}_trace.xlsx: {e}")
        continue
    
    low_list = low['video_name'].str.replace('.mp4', '', regex=False)
    
    # Store predictions per model
    model_preds = {}
    
    for model in model_names:
        try:
            df = pd.read_csv(f'../model_predictions/vad_results_{model}.csv')
            df['Video Name'] = df['Video Name'].str.replace('.jpg', '', regex=False)
            df = df[df['Video Name'].isin(low_list)]
            model_preds[model] = df[['Video Name', 'Predicted Label']].set_index('Video Name')
        except Exception as e:
            print(f"[Error] Failed to process {model}: {e}")
    
    # Merge all predictions
    merged = pd.DataFrame(index=low_list)
    for model in model_names:
        merged[model] = model_preds.get(model, pd.DataFrame()).reindex(low_list)['Predicted Label']
    
    # Drop rows with missing predictions
    merged = merged.dropna()
    
    if merged.empty:
        print(f"[Warning] No valid data for P={P}, skipping.")
        continue
    
    # Load ground truth from claude-3-5-sonnet
    gt_df = pd.read_csv('../model_predictions/vad_results_claude3.5sonnet.csv')
    gt_df['Video Name'] = gt_df['Video Name'].str.replace('.mp4', '', regex=False)
    gt_df = gt_df[gt_df['Video Name'].isin(merged.index)]
    ground_truth = gt_df.set_index('Video Name')['True Label']
    
    # Majority voting
    majority_vote = merged.mode(axis=1)[0]
    
    # Align ground truth
    y_true = ground_truth.loc[merged.index]
    
    # Compute metrics
    acc = accuracy_score(y_true, majority_vote)
    prec = precision_score(y_true, majority_vote,  average='macro',zero_division=0)
    rec = recall_score(y_true, majority_vote, average='macro', zero_division=0)
    f1 = f1_score(y_true, majority_vote, average='macro', zero_division=0)
    
    all_results.append({
        'P': P,
        'Num Videos': len(merged),
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1
    })
    
    print(f"[P={P}] Processed {len(merged)} videos | Accuracy: {acc:.4f}, F1-Score: {f1:.4f}")

# Save final results to one Excel file
result_df = pd.DataFrame(all_results)
result_df.to_excel(f'low list/VAD_Majority_Voting_Summary_normalized_meansigma_bestalpha_average_5.xlsx', index=False)

print("[Saved] VAD_Majority_Voting_Summary.xlsx")


In [ ]:
import pandas as pd




# Load the summary files
majority_df = pd.read_excel(f'low list/VAD_Majority_Voting_Summary_normalized_meansigma_bestalpha_average_5.xlsx')

# Extract metrics
P_values = majority_df['P']
overall_accuracy = majority_df['Accuracy']
# recall = majority_df['Recall']
# vague_abnormal_accuracy = vague_df['Accuracy']

# Combine into final DataFrame
out_df = pd.DataFrame({
    'P': P_values,
    'Overall Accuracy': overall_accuracy
    # 'Recall': recall,
    # 'Vague Abnormal Accuracy': vague_abnormal_accuracy
})

# Remove final two rows
out_df = out_df.iloc[:, :]

# Save to xlsx
out_df.to_excel(f'low list/results_P_normalized_meansigma_bestalpha_average_5.xlsx', index=False)
